In [1]:
# Set up and load data
# Includes
import sys
import os
import numpy as np
import json
import os


# Setup paths containing utility
curr_folder = os.getcwd()
sys.path.insert(0, os.path.join(curr_folder,'../../../app'))

# Utils imports for loading data
from utils import save_data, load_data, exists_datafolder
from utils import load_SQuAD_train, load_SQuAD_dev
from utils import get_foldername
from utils_NLP import text2sentences,words2words_blanked,words2words_hashblank,words2answers
from utils_NLP import words2text
from utils_SQuAD import OR_arts_paragraph_fields,merge_arts_paragraph_fields
from utils_NLP import allenNLP_classify_blanks,allenNLP_classify_blanks_fromResults

# Plotting includes
from utils_EDAplots import plotbar_train_dev,plothist_train_dev,plotbar_train_dev2,plothist_train_dev2

# Stats saving stuff
from utils_EDA import calcstats_train_dev

# AllenNLP stuff
from allennlp.predictors import Predictor

# Include custom AllenNLP
import myallennlp
from myallennlp import *
from myallennlp.models.simple_tagger2 import SimpleTagger2
from myallennlp.dataset_readers import sequence_tagging2
from myallennlp.data.tokenizers.word_splitter import SpacyWordSplitter

# Import fig stuff
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
# Option for merging NER data into combined model
merge_in_NER_data = False

In [3]:
!pwd

/home/davestanley/Dropbox/git/mindpocket/train_model/articles_batch5.0/entropy_90_10_model6e0c37


# Manually test the predictor

## Load the predictor

In [4]:
# Set up AllenNLP
currmodel = os.path.join('.','model.tar.gz')
predictor = Predictor.from_path(currmodel,predictor_name='sentence-tagger');

02/25/2019 23:56:32 - INFO - allennlp.models.archival -   loading archive file ./model.tar.gz
02/25/2019 23:56:32 - INFO - allennlp.models.archival -   extracting archive file ./model.tar.gz to temp dir /tmp/tmpkipg_by4
02/25/2019 23:56:32 - INFO - allennlp.common.params -   type = default
02/25/2019 23:56:32 - INFO - allennlp.data.vocabulary -   Loading token dictionary from /tmp/tmpkipg_by4/vocabulary.
02/25/2019 23:56:32 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.models.model.Model'> from params {'Ntags0': 90, 'Ntags1': 10, 'do_crossentropy_weighting': True, 'encoder': {'bidirectional': True, 'dropout': 0.5, 'hidden_size': 72, 'input_size': 72, 'num_layers': 2, 'type': 'lstm'}, 'text_field_embedder': {'token_embedders': {'dependency_label': {'embedding_dim': 10, 'type': 'embedding', 'vocab_namespace': 'dependencies'}, 'ner_tag': {'embedding_dim': 7, 'type': 'embedding', 'vocab_namespace': 'ner'}, 'pos_tag': {'embedding_dim': 5, 'type': 'embedding',

## Run on test sentence

In [5]:
results = predictor.predict(sentence='Pitch is an auditory sensation based on the frequency of vibration.');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_hashblank(words,tags)
print(out)

['___Pitch___', 'is', 'an', '___auditory___', 'sensation', 'based', 'on', 'the', 'frequency', 'of', '___vibration___', '.']


In [6]:
results = predictor.predict(sentence='The Normans are from northern France.');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_hashblank(words,tags)
print(out)

['The', '___Normans___', 'are', 'from', 'northern', '___France___', '.']


In [7]:
results = predictor.predict(sentence='The brain is an organ that serves as the center of the nervous system in all vertebrate and most invertebrate animals.');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_hashblank(words,tags)
print(out)

['___The___', '___brain___', 'is', 'an', 'organ', 'that', 'serves', 'as', 'the', 'center', 'of', 'the', 'nervous', 'system', 'in', 'all', '___vertebrate___', 'and', 'most', 'invertebrate', 'animals', '.']


In [8]:
results = predictor.predict(sentence='Physiologically, the function of the brain is to exert centralized control over the other organs of the body.');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', ',', 'the', 'function', 'of', 'the', '______', 'is', 'to', 'exert', '______', 'control', 'over', 'the', 'other', 'organs', 'of', 'the', 'body', '.']


In [9]:
results = predictor.predict(sentence='The Black Death ravaged Europe for three years before it continued on into Russia, where the disease was present somewhere in the country 25 times between 1350 to 1490.');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['The', 'Black', 'Death', 'ravaged', 'Europe', 'for', 'three', 'years', 'before', 'it', 'continued', 'on', 'into', '______', ',', 'where', 'the', 'disease', 'was', 'present', 'somewhere', 'in', 'the', 'country', '______', '______', 'between', '______', 'to', '______', '.']


In [10]:
results = predictor.predict(sentence='The Black Death ravaged Europe for three years before it continued on into Russia.');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['The', 'Black', 'Death', 'ravaged', 'Europe', 'for', 'three', 'years', 'before', 'it', 'continued', 'on', 'into', '______', '.']


In [11]:
results = predictor.predict(sentence='I continued on into Russia, which is a country.');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['I', 'continued', 'on', 'into', '______', ',', 'which', 'is', 'a', 'country', '.']


In [12]:
results = predictor.predict(sentence='I continued on into Russia, which is a food.');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['I', 'continued', 'on', 'into', '______', ',', 'which', 'is', 'a', 'food', '.']


In [13]:
results = predictor.predict(sentence='Moscow is the capital of Russia.');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'the', 'capital', 'of', '______', '.']


In [14]:
results = predictor.predict(sentence='London is the capital of England.');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'the', 'capital', 'of', '______', '.']


In [15]:
results = predictor.predict(sentence='Berlin is the capital of England');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'the', 'capital', 'of', '______']


In [16]:
results = predictor.predict(sentence='Plague is a type of disease');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'a', 'type', 'of', 'disease']


In [17]:
# # Pull out sample paragraph
# p = arts[ind_ex_dev]['paragraphs'][0]

# # # Print all AllenNLP classifications
# # print([(a,b) for a,b in zip(p['allenNER']['words'].split(), p['allenNER']['tags'].split())])

# # AllenNLP results
# words = p['allenNER']['words'].split()
# # tags = p['allenNER']['tags'].split()
# # tags = [not t == '0' for t in tags]   # Convert to binary
# tags = p['blank_classified_allen']

# # Ground truth
# blank_classification = p['blank_classification']


# words_blanked_ground_truth = words2words_blanked(words,blank_classification)
# words_blanked_allen = words2words_blanked(words,tags)


In [18]:
results = predictor.predict(sentence='Tea is eaten with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'eaten', 'with', '______']


In [19]:
results = predictor.predict(sentence='Pastry is eaten with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'eaten', 'with', '______']


In [20]:
results = predictor.predict(sentence='Cake is eaten with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'eaten', 'with', 'crumpets']


In [21]:
results = predictor.predict(sentence='Beef is eaten with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['Beef', 'is', 'eaten', 'with', 'crumpets']


In [22]:
results = predictor.predict(sentence='Corn is eaten with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'eaten', 'with', 'crumpets']


In [23]:
results = predictor.predict(sentence='Milk is eaten with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'eaten', 'with', 'crumpets']


In [24]:
results = predictor.predict(sentence='Juice is eaten with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'eaten', 'with', 'crumpets']


In [25]:
results = predictor.predict(sentence='England is eaten with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'eaten', 'with', '______']


In [26]:
results = predictor.predict(sentence='Tea is eaten with with with with with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', '______', 'with', 'with', 'with', 'with', 'with', '______']


In [27]:
results = predictor.predict(sentence='England is eaten with with with with with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', 'eaten', 'with', 'with', 'with', 'with', 'with', '______']


In [28]:
results = predictor.predict(sentence='Beef is eaten with with with with with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['Beef', 'is', '______', 'with', 'with', 'with', 'with', 'with', 'crumpets']


In [29]:
results = predictor.predict(sentence='Corn is eaten with with with with with crumpets');
words = results['words']
tags = allenNLP_classify_blanks_fromResults(results,'0')
out = words_blanked_allen = words2words_blanked(words,tags)
print(out)

['______', 'is', '______', 'with', 'with', 'with', 'with', 'with', 'crumpets']
